In [2]:
import pandas as pd

In [4]:
from binance.client import Client

In [8]:
from datetime import datetime, timedelta
import time

In [9]:
# Define Binance API keys
api_key = "vVUjcdC5FcMDpgK3AUHfWITEAfelYUNCMlrejrb5STLH2gbzveK5GkBzxX8lEYtv"
api_secret = "id5sMkBu6h4AfLo0VWYKGweeTRtbt7QLRPIY4z9ks7IpFqPruEAsahZ4PDnmZLh6"

# Initialize Binance client
client = Client(api_key, api_secret)

In [13]:

# Define the symbols and interval
symbols = ["SHIBUSDT","BTCUSDT"]  # Added SHIBA (SHIB)
interval = Client.KLINE_INTERVAL_15MINUTE  # Set the interval to 15 minutes

# Desired row count
desired_rows = 60000  # Fetch up to 60,000 rows

# Create an empty dictionary to store data
data_dict = {}

# Loop through each symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    
    # Initialize an empty DataFrame to store the data for this symbol
    all_data = pd.DataFrame()

    # First fetch the most recent 15-minute candlestick to determine the start time
    klines = client.get_historical_klines(symbol, interval, "1 day ago UTC", limit=1)
    columns = [
        "Open time", "Open", "High", "Low", "Close", "Volume",
        "Close time", "Quote asset volume", "Number of trades",
        "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"
    ]
    df = pd.DataFrame(klines, columns=columns)
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms")
    df["Close time"] = pd.to_datetime(df["Close time"], unit="ms")
    df = df.drop(columns=["Ignore"])  # Drop unnecessary column
    
    # Set the start time as the most recent "Close time"
    start_time = df["Close time"].iloc[-1]

    rows_fetched = 0
    
    # Keep fetching data until the desired number of rows is reached
    while rows_fetched < desired_rows:
        # Fetch data in chunks (500 rows per request)
        klines = client.get_historical_klines(
            symbol, 
            interval, 
            start_time.strftime("%Y-%m-%d %H:%M:%S"),  # Start from the last Close time
            limit=500  # Limit to 500 data points per request
        )
        
        # Check if any data was returned
        if not klines:
            print(f"No data returned for {symbol} at {start_time}. Skipping.")
            break
        
        # Create a DataFrame for the current chunk
        df = pd.DataFrame(klines, columns=columns)
        df["Open time"] = pd.to_datetime(df["Open time"], unit="ms")
        df["Close time"] = pd.to_datetime(df["Close time"], unit="ms")
        df = df.drop(columns=["Ignore"])  # Drop unnecessary column
        
        # Check if the DataFrame is not empty
        if not df.empty:
            # Append the new data to the main DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)
            
            # Update the number of rows fetched
            rows_fetched = len(all_data)
            print(f"Fetched {rows_fetched} rows for {symbol}")
            
            # Update the start_time for the next chunk of data (go backwards)
            start_time = df["Open time"].iloc[0]  # Set start_time to the first Open time of the chunk
        else:
            print(f"Skipping empty data for {symbol}.")
            break
        
        # Sleep to avoid hitting API rate limits
        time.sleep(1)  # Adjust as needed to avoid rate limit issues

    # Store the data for this symbol if data was fetched
    if not all_data.empty:
        data_dict[symbol] = all_data
        # Save each crypto's data to a CSV file
        all_data.to_csv(f"{symbol}_15min_data.csv", index=False)

# Combine all crypto data into a single DataFrame
if data_dict:
    combined_data = pd.concat(data_dict.values(), axis=1, keys=data_dict.keys())

    # Save combined data to a CSV
    combined_data.to_csv("multiple_cryptos_combined_15min.csv")

    # Print task completion message
    print("Task Completed: Data fetch for all symbols (15-minute interval) is done and saved.")
else:
    print("No data was fetched for any symbols.")


Fetching data for SHIBUSDT...
Fetched 94 rows for SHIBUSDT
Fetched 188 rows for SHIBUSDT
Fetched 282 rows for SHIBUSDT
Fetched 376 rows for SHIBUSDT
Fetched 471 rows for SHIBUSDT
Fetched 566 rows for SHIBUSDT
Fetched 661 rows for SHIBUSDT
Fetched 756 rows for SHIBUSDT
Fetched 851 rows for SHIBUSDT
Fetched 946 rows for SHIBUSDT
Fetched 1041 rows for SHIBUSDT
Fetched 1136 rows for SHIBUSDT
Fetched 1231 rows for SHIBUSDT
Fetched 1326 rows for SHIBUSDT
Fetched 1421 rows for SHIBUSDT
Fetched 1516 rows for SHIBUSDT
Fetched 1611 rows for SHIBUSDT
Fetched 1706 rows for SHIBUSDT
Fetched 1801 rows for SHIBUSDT
Fetched 1896 rows for SHIBUSDT
Fetched 1991 rows for SHIBUSDT
Fetched 2086 rows for SHIBUSDT
Fetched 2181 rows for SHIBUSDT
Fetched 2276 rows for SHIBUSDT
Fetched 2371 rows for SHIBUSDT
Fetched 2466 rows for SHIBUSDT
Fetched 2561 rows for SHIBUSDT
Fetched 2656 rows for SHIBUSDT
Fetched 2751 rows for SHIBUSDT
Fetched 2846 rows for SHIBUSDT
Fetched 2941 rows for SHIBUSDT
Fetched 3036 rows fo